In [ ]:
# Solution Ex. 46 SparkSQL - V1

In [ ]:
import sys

In [ ]:
inputPath = "/data/students/bigdata-01QYD/ex_data/Ex46/data/readings.txt"
outputPath = "res_out_Ex46SparkSQLV1/"

In [ ]:
# Read the content of readings
# Create a DataFrame from persons.csv
readingsDF = spark.read.load(inputPath,\
                             format="csv",\
                             header=False,\
                             inferSchema=True)\
.withColumnRenamed("_c0","timestamp")\
.withColumnRenamed("_c1","temperature")

In [ ]:
#debug
readingsDF.show()
#readingsDF.printSchema()

In [ ]:
# Transform Datafarme into an RDD
# The RDD is an RDD of Row objects
readingsRDD = readingsDF.rdd;

In [ ]:
#debug
readingsRDD.collect()

In [ ]:
# Generate the elements of each window.
# Each reading with start time t belongs to 3 windows with a window size equal to 3:
# - The one starting at time t-120s
# - The one starting at time t-60s
# - The one starting at time t

def windowElementsFunc(reading):

    # Time stamp of this reading
    t = reading['timestamp']
    # Temperature
    temperature = reading['temperature']
    
    # The current reading, associated with time stamp t,
    # is part of the windows starting at time t, t-60s, t-120s
    
    # pairs is a list containing three pairs (window start timestamp, current reading) associated with
    # the three windows containing this reading
    pairs = []
    
    # Window starting at time t
    # This reading is the first element of the window starting at time t
    pairs.append((t, reading))
    
    # Window starting at time t-60
    # This reading is the second element of that window starting at time t-60
    pairs.append((t-60, reading))

    # Window starting at time t-120
    # This reading is the third element of that window starting at time t-120
    pairs.append((t-120, reading))

    return pairs

In [ ]:
windowsElementsRDD = readingsRDD\
.flatMap(windowElementsFunc)

In [ ]:
#debug
windowsElementsRDD.collect()

In [ ]:
# Use groupByKey to generate one sequence for each time stamp
timestampsWindowsRDD = windowsElementsRDD.groupByKey()

In [ ]:
#debug
timestampsWindowsRDD.mapValues(lambda v: list(v)).collect()

In [ ]:
# Select the values of timestampsWindowsRDD. The key is useless for the next steps.
# Each value is one window composed of at most 3 elements/readings "time stamp,temperature"
windowsRDD =  timestampsWindowsRDD.values()

In [ ]:
#debug
windowsRDD.map(lambda v: list(v)).collect()

In [ ]:
# This function is used in the next transformation to select the windows with an incrasing temperature trend
def increasingTrendFunc(window):
    # Store the (at most) 3 elements of the window in a dictionary
    # containing enties time stamp -> temperature
    timestampTemp = {}

    # Compute also the information about minimum time stamp, i.e., the first timestamp of the window
    minTimestamp = sys.maxsize

    for timestampTemperature in window:
        t = timestampTemperature['timestamp']
        temperature = timestampTemperature['temperature']
        
        timestampTemp[t] = temperature
        
        if t<minTimestamp:
            minTimestamp=t
    
    # Check if the list contains three elements.
    # If the number of elements is not equal to 3 the window is incomplete and must be discarded
    if len(timestampTemp) != 3:
        increasing = False
    else:
        # Check if the increasing trend is satisfied
        if timestampTemp[minTimestamp]<timestampTemp[minTimestamp+60] and timestampTemp[minTimestamp+60]<timestampTemp[minTimestamp+120]:
            increasing = True
        else:
            increasing = False

    return increasing

In [ ]:
seletedWindowsRDD = windowsRDD\
.filter(increasingTrendFunc)

In [ ]:
# Debug
seletedWindowsRDD.map(lambda v: list(v)).collect()

In [ ]:
# Store the result. Map the iterable associated with each window to a list

In [ ]:
seletedWindowsRDD\
.map(lambda window: list(window))\
.saveAsTextFile(outputPath+"Bis")